In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request

In [ ]:
def get_ticks(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'page')
    
    ticks = list()
    stocks = list()
    for name_box in news_table.find_all('a', href = True):
        stocks.append(name_box.text.strip())
        ticks.append(name_box['href'].split('-')[-1].strip())
    d = {'stock':stocks, 'tick':ticks}
    df = pd.DataFrame(data = d)
    return df

In [ ]:
url = "https://www.tickertape.in/stocks"

tick_df = get_ticks(url)

In [ ]:
tick_df

In [ ]:
def get_data(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req).read()    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'latest-news-holder')
    
    news = list()
    
    for name_box in news_table.find_all('p', class_='shave-root'):
        news.append(name_box.text.strip())
    
    return news

In [ ]:
news = list()
for i, tick in enumerate(tick_df['tick']):
    url = "https://www.tickertape.in/stocks/" + tick + "/news?checklist=basic&ref=stock-overview_overview-sections&type=news"
    headlines = get_data(url)
    news.append(headlines)

In [ ]:
for i, n in enumerate(news):
    print(i+1, ")", n)
    print('-------')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [ ]:
def detect(news):
    tot_val = list()
    for n in news:
        if len(n) == 0:
            tot_val.append(['neutral'])
        else:
            inputs = tokenizer(n, return_tensors="pt", padding=True)
            outputs = finbert(**inputs)[0]
            val = list()
            for idx, sent in enumerate(n):
                val.append(labels[np.argmax(outputs.detach().numpy()[idx])])
                print(sent, '----', val[idx])
            print('#######################################################')    
            tot_val.append(val)
    return tot_val

In [ ]:
tot_val = detect(news)

print(tot_val)

In [ ]:
def get_sent(val):
    agg = 0
    for i in val:
        if i == 'positive':
            agg = agg + 1
        elif i == 'negative':
            agg = agg - 1

    if agg > 0:
        return('positive')
    elif agg < 0:
        return('negative')
    else:
        return('neutral')

In [ ]:
sent = list()
for i in tot_val:
    sent.append(get_sent(i))

In [ ]:
sent

In [ ]:
tick_df['sentiment'] = sent 

tick_df.head(20)